### Importing Library

In [ ]:
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)

In [ ]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models
from tensorflow.keras import Model 


### Explore The Dataset 


In [ ]:
dataset_dir = "/kaggle/input/plantvillage-dataset/color"


##### change this current directory to this path,


In [ ]:
cd /kaggle/input/plantvillage-dataset


In [ ]:
print(os.listdir("plantvillage dataset"))


print(len(os.listdir("plantvillage dataset/segmented")))
print(os.listdir("plantvillage dataset/segmented")[:5])

print(len(os.listdir("plantvillage dataset/color")))
print(os.listdir("plantvillage dataset/color")[:5])

print(len(os.listdir("plantvillage dataset/grayscale")))
print(os.listdir("plantvillage dataset/grayscale")[:5])

In [ ]:
print(len(os.listdir("plantvillage dataset/color/Grape___healthy")))


#### Data preprocessing

In [ ]:
#Dataset Path
dataset_dir = "/kaggle/input/plantvillage-dataset/color"


In [ ]:
image_path='/kaggle/input/plantvillage-dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'
img=mpimg.imread(image_path)
print(img.shape)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
image_path='/kaggle/input/plantvillage-dataset/grayscale/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'
img=mpimg.imread(image_path)
print(img.shape)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
image_path='/kaggle/input/plantvillage-dataset/segmented/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003_final_masked.jpg'
img=mpimg.imread(image_path)
print(img.shape)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
image_path = '/kaggle/input/plantvillage-dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'

# Read the image(Pixels)
img = mpimg.imread(image_path)

print(img)

In [ ]:
# Image Parameters
img_size = 224
batch_size = 32

#### Train Test Split

In [ ]:
# Image Data Generators
data_gen=ImageDataGenerator(
rescale=1./255,
validation_split=0.2, 
rotation_range=0.2,
horizontal_flip=0.2,   # Randomly flip images horizontally
vertical_flip=0.2 
)

In [ ]:
# Train Generator
train_generator=data_gen.flow_from_directory(
dataset_dir,
target_size=(img_size,img_size),
batch_size=batch_size,
subset='training',
class_mode='categorical'
)

In [ ]:
validation_generator=data_gen.flow_from_directory(
dataset_dir,
target_size=(img_size,img_size),
batch_size=batch_size,
subset='validation',
class_mode='categorical'
)

### Convolutional Neural Network

In [ ]:
model = models.Sequential([
    layers.Conv2D(32,(3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
# Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,  # Number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size       )

In [ ]:
# Model Evaluation
print("Evaluating model...")
val_loss, val_accuracy = model_final.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Function to Load and Preprocess the Image using Pillow
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    img = Image.open(image_path)
    # Resize the image
    img = img.resize(target_size)
    # Convert the image to a numpy array
    img_array = np.array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Scale the image values to [0, 1]
    img_array = img_array.astype('float32') / 255.
    return img_array

# Function to Predict the Class of an Image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [ ]:
# Create a mapping from class indices to class names
class_indices = {v: k for k, v in train_generator.class_indices.items()}

In [ ]:
class_indices

In [ ]:
image_path='/kaggle/input/plantvillage-dataset/color/Strawberry___healthy/00166615-5e7b-4318-8957-5e50df335ee8___RS_HL 1785.JPG'
predicted_class_name = predict_image_class(model, image_path, class_indices)
img=mpimg.imread(image_path)
plt.imshow(img)
# Output the result
print("Predicted Class Name:", predicted_class_name)